# M2177.003100 Deep Learning <br> Assignment #3 Part 3: Language Modeling with CharRNN

Copyright (C) Data Science Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Sang-gil Lee, October 2018

This is a character-level language model using recurrent neural networks (RNNs).
It has become very popular as a starter kit for learning how RNN works in practice.

Before we start, what is "language modeling" anyway? Intuitively, "language modeling" is teaching the model about a general probability distribution of our words and sentences.

So we ask the model like: "hey just say whatever words from your estimation of the wikipedia word distribution", and the model responds like "ok, i learned from wikipedia, and the most frequent word is "the". so let me start with "the". the wikipedia is blah blah blah"

Thus, by teaching the model to speak for itself, we can test the model's capability of learning temporal relationships between sequences.

Original blog post & code:
https://github.com/karpathy/char-rnn
http://karpathy.github.io/2015/05/21/rnn-effectiveness/

But the original code is written in lua torch which looks less pretty :(

There is a clean port of char-RNN in TensorFlow
https://github.com/sherjilozair/char-rnn-tensorflow
This iPython notebook is basically a copypasta of this repo.

That said, you are allowed to copy paste the codes from the original repo.
HOWEVER, <font color=red> try to implement the model yourself first </font>, and consider the original source code as a last resort.
You will learn a lot while wrapping around your head during the implementation. And you will understand nuts and bolts of RNNs more clearly in a code level.

### AND MOST IMPORTANTLY, IF YOU JUST BLINDLY COPY PASTE THE CODE, YOU SHALL RUIN YOUR EXAM.
### The exam is designed to be solvable for students that actually have written the code themselves.
At least strictly re-type the codes from the original repo line-by-line, and understand what each line means thoroughly.

## YOU HAVE BEEN WARNED. :)



### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **all Assignment Part 1-5**, run the *CollectSubmission.sh* script with your **Team number** as input argument. <br>
This will produce a zipped file called *[Your team number].zip*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* team_#)

### Character language modeling (20 points)

This assignment is an on/off one: just make this notebook **"work"** without problem by: 

1. implementing **1. \_\_init\_\_()** and **2. sample()** of RNN **Model()** class from **char_rnn.py**

2. briefly summarizing, at the end of the script, how you implmeneted the model & why you changed some other parts of the code. yes,  <font color=red> there are other intentional pitfalls inside the code </font>. just copy-pasting the \_\_init\_\_() will not work. can you tell me why?

### The Definition of **"work"** is as follows:

1. Training loss must be <font color=red> below 0.2 </font>. We will check the training loss output from the training code block. We don't split the data into train-valid-test. Don't forget to <font color=red> NOT clear the output from train(args)</font>, where the training loss will be printed! TA will check the logged output from train(args)

2. calling sample(args.sample) at the last code block <font color=red> must generate some meaningful sentences </font>. The quality of the sentence does not count, unless the generated sentence is something like "aaaaaaaaaaaaaabbbbbb" or "b" u tlttfcwaU c  fGcnrh i.\nh mt he!bsthpme".



Now proceed to the code.

In [1]:
# ipython magic function for limiting the gpu to be seen for tensorflow
# if you have just 1 GPU, specify the value to 0
# if you have multiple GPUs (nut) and want to specify which GPU to use, specify this value to 0 or 1 or etc.
%env CUDA_DEVICE_ORDER = PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES = 1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
# load a bunch of libraries
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.contrib import legacy_seq2seq
import numpy as np
import argparse
import time
import os
from six.moves import cPickle
from six import text_type
import sys

# this module is from the utils.py file of this folder
# it handles loading texts to digits (aka. tokens) which are recognizable for the model
from utils import TextLoader

# this module is from the char_rnn.py file of this folder
# the task is implementing the CharRNN inside the class definition from this file
from char_rnn import Model

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
# for TensorFlow vram efficiency: if this is not specified, the model hogs all the VRAM even if it's not necessary
# bad & greedy TF! but it has a reason for this design choice FWIW, try googling it if interested
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [4]:
# argparsing
parser = argparse.ArgumentParser(
                    formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# Data and model checkpoints directories
parser.add_argument('--data_dir', type=str, default='data/tinyshakespeare',
                    help='data directory containing input.txt with training examples')
parser.add_argument('--save_dir', type=str, default='models_char_rnn',
                    help='directory to store checkpointed models')
parser.add_argument('--save_every', type=int, default=1000,
                    help='Save frequency. Number of passes between checkpoints of the model.')
parser.add_argument('--init_from', type=str, default=None,
                    help="""continue training from saved model at this path (usually "save").
                        Path must contain files saved by previous training process:
                        'config.pkl'        : configuration;
                        'chars_vocab.pkl'   : vocabulary definitions;
                        'checkpoint'        : paths to model file(s) (created by tf).
                                              Note: this file contains absolute paths, be careful when moving files around;
                        'model.ckpt-*'      : file(s) with model definition (created by tf)
                         Model params must be the same between multiple runs (model, rnn_size, num_layers and seq_length).
                    """)
# Model params
parser.add_argument('--model', type=str, default='lstm',
                    help='lstm, rnn, gru, or nas')
parser.add_argument('--rnn_size', type=int, default=128,
                    help='size of RNN hidden state')
parser.add_argument('--num_layers', type=int, default=5,
                    help='number of layers in the RNN')
# Optimization
parser.add_argument('--seq_length', type=int, default=500,
                    help='RNN sequence length. Number of timesteps to unroll for.')
parser.add_argument('--batch_size', type=int, default=5,
                    help="""minibatch size. Number of sequences propagated through the network in parallel.
                            Pick batch-sizes to fully leverage the GPU (e.g. until the memory is filled up)
                            commonly in the range 10-500.""")
parser.add_argument('--num_epochs', type=int, default=50,
                    help='number of epochs. Number of full passes through the training examples.')
parser.add_argument('--grad_clip', type=float, default=20.,
                    help='clip gradients at this value')
parser.add_argument('--learning_rate', type=float, default=0.0002,
                    help='learning rate')
parser.add_argument('--decay_rate', type=float, default=0.97,
                    help='decay rate for rmsprop')
parser.add_argument('--output_keep_prob', type=float, default=1.0,
                    help='probability of keeping weights in the hidden layer')
parser.add_argument('--input_keep_prob', type=float, default=1.0,
                    help='probability of keeping weights in the input layer')

# needed for argparsing within jupyter notebook
# https://stackoverflow.com/questions/30656777/how-to-call-module-written-with-argparse-in-ipython-notebook
sys.argv = ['-f']
args = parser.parse_args()

# print args: see if the hyperparemeters look pretty to you
args

Namespace(batch_size=5, data_dir='data/tinyshakespeare', decay_rate=0.97, grad_clip=20.0, init_from=None, input_keep_prob=1.0, learning_rate=0.0002, model='lstm', num_epochs=50, num_layers=5, output_keep_prob=1.0, rnn_size=128, save_dir='models_char_rnn', save_every=1000, seq_length=500)

In [5]:
# protip: always check the data and poke around the data yourself
# you will get a lot of insights by looking at the data
data_loader = TextLoader(args.data_dir, args.batch_size, args.seq_length)
data_loader.reset_batch_pointer()
args.vocab_size = data_loader.vocab_size

x, y = data_loader.next_batch()

# our data has a shape of (N, T), where N=batch_size and T=seq_length
print(x)
print(x.shape)
print(y)
print(y.shape)

loading preprocessed files
[[49  9  7 ...  3  3  7]
 [11  1  6 ... 15  0 17]
 [26  0 19 ... 18  0  2]
 [ 8  0  3 ...  2  0  6]
 [18  9  2 ... 21  0  5]]
(5, 500)
[[ 9  7  6 ...  3  7  0]
 [ 1  6  0 ...  0 17  1]
 [ 0 19  7 ...  0  2  3]
 [ 0  3 13 ...  0  6  4]
 [ 9  2  9 ...  0  5  4]]
(5, 500)


In [6]:
# see what the first entry of the batch look like
print(x[0])
print(y[0])
# y is just an x shifted to the left by one: so the network will predict the next token y given x. 

[49  9  7  6  2  0 37  9  2  9 57  1  8 24 10 43  1 18  3  7  1  0 17  1
  0 23  7  3 19  1  1 12  0  4  8 15  0 18 13  7  2  5  1  7 16  0  5  1
  4  7  0 14  1  0  6 23  1  4 28 25 10 10 26 11 11 24 10 35 23  1  4 28
 16  0  6 23  1  4 28 25 10 10 49  9  7  6  2  0 37  9  2  9 57  1  8 24
 10 50  3 13  0  4  7  1  0  4 11 11  0  7  1  6  3 11 27  1 12  0  7  4
  2  5  1  7  0  2  3  0 12  9  1  0  2  5  4  8  0  2  3  0 18  4 14  9
  6  5 44 10 10 26 11 11 24 10 34  1  6  3 11 27  1 12 25  0  7  1  6  3
 11 27  1 12 25 10 10 49  9  7  6  2  0 37  9  2  9 57  1  8 24 10 49  9
  7  6  2 16  0 15  3 13  0 28  8  3 17  0 37  4  9 13  6  0 42  4  7 19
  9 13  6  0  9  6  0 19  5  9  1 18  0  1  8  1 14 15  0  2  3  0  2  5
  1  0 23  1  3 23 11  1 25 10 10 26 11 11 24 10 39  1  0 28  8  3 17 30
  2 16  0 17  1  0 28  8  3 17 30  2 25 10 10 49  9  7  6  2  0 37  9  2
  9 57  1  8 24 10 36  1  2  0 13  6  0 28  9 11 11  0  5  9 14 16  0  4
  8 12  0 17  1 30 11 11  0  5  4 27  1  0 19  3  7

In [7]:
# training loop definition
def train(args):
    data_loader = TextLoader(args.data_dir, args.batch_size, args.seq_length)
    args.vocab_size = data_loader.vocab_size
    print("vocabulary size: " + str(args.vocab_size))

    # check compatibility if training is continued from previously saved model
    if args.init_from is not None:
        # check if all necessary files exist
        assert os.path.isdir(args.init_from)," %s must be a a path" % args.init_from
        assert os.path.isfile(os.path.join(args.init_from,"config.pkl")),"config.pkl file does not exist in path %s"%args.init_from
        assert os.path.isfile(os.path.join(args.init_from,"chars_vocab.pkl")),"chars_vocab.pkl.pkl file does not exist in path %s" % args.init_from
        ckpt = tf.train.latest_checkpoint(args.init_from)
        assert ckpt, "No checkpoint found"

        # open old config and check if models are compatible
        with open(os.path.join(args.init_from, 'config.pkl'), 'rb') as f:
            saved_model_args = cPickle.load(f)
        need_be_same = ["model", "rnn_size", "num_layers", "seq_length"]
        for checkme in need_be_same:
            assert vars(saved_model_args)[checkme]==vars(args)[checkme],"Command line argument and saved model disagree on '%s' "%checkme

        # open saved vocab/dict and check if vocabs/dicts are compatible
        with open(os.path.join(args.init_from, 'chars_vocab.pkl'), 'rb') as f:
            saved_chars, saved_vocab = cPickle.load(f)
        assert saved_chars==data_loader.chars, "Data and loaded model disagree on character set!"
        assert saved_vocab==data_loader.vocab, "Data and loaded model disagree on dictionary mappings!"

    if not os.path.isdir(args.save_dir):
        os.makedirs(args.save_dir)
    with open(os.path.join(args.save_dir, 'config.pkl'), 'wb') as f:
        cPickle.dump(args, f)
    with open(os.path.join(args.save_dir, 'chars_vocab.pkl'), 'wb') as f:
        cPickle.dump((data_loader.chars, data_loader.vocab), f)
    
    print("building the model... may take some time...")
    ##################### This line builds the CharRNN model defined in char_rnn.py #####################
    model = Model(args)
    print("model built! starting training...")

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)
        # restore model
        if args.init_from is not None:
            saver.restore(sess, ckpt)
        for e in range(args.num_epochs):
            sess.run(tf.assign(model.lr,
                               args.learning_rate * (args.decay_rate ** e)))
            data_loader.reset_batch_pointer()
            state = sess.run(model.initial_state)
            
            for b in range(int(data_loader.num_batches)):
                start = time.time()
                x, y = data_loader.next_batch()
                feed = {model.input_data: x, model.targets: y}
                for i, (c, h) in enumerate(model.initial_state):
                    feed[c] = state[i].c
                    feed[h] = state[i].h

                train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)

                end = time.time()
                
                # print training log every 100 steps
                if ((e * data_loader.num_batches + b) % 100 == 0):
                    print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}"
                          .format(e * data_loader.num_batches + b,
                                  args.num_epochs * data_loader.num_batches,
                                  e, train_loss, end - start))
                if (e * data_loader.num_batches + b) % args.save_every == 0\
                        or (e == args.num_epochs-1 and
                            b == data_loader.num_batches-1):
                    # save for the last result
                    checkpoint_path = os.path.join(args.save_dir, 'model.ckpt')
                    saver.save(sess, checkpoint_path,
                               global_step=e * data_loader.num_batches + b)
                    print("model saved to {}".format(checkpoint_path))


In [8]:
os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [9]:
# let's train!
train(args)

loading preprocessed files
vocabulary size: 65
building the model... may take some time...
model built! starting training...
0/22300 (epoch 0), train_loss = 4.175, time/batch = 6.283
model saved to models_char_rnn/model.ckpt
100/22300 (epoch 0), train_loss = 3.342, time/batch = 0.331
200/22300 (epoch 0), train_loss = 3.289, time/batch = 0.334
300/22300 (epoch 0), train_loss = 3.304, time/batch = 0.335
400/22300 (epoch 0), train_loss = 3.284, time/batch = 0.328
500/22300 (epoch 1), train_loss = 3.284, time/batch = 0.334
600/22300 (epoch 1), train_loss = 3.345, time/batch = 0.333
700/22300 (epoch 1), train_loss = 3.238, time/batch = 0.331
800/22300 (epoch 1), train_loss = 3.298, time/batch = 0.356
900/22300 (epoch 2), train_loss = 3.316, time/batch = 0.333
1000/22300 (epoch 2), train_loss = 3.332, time/batch = 0.334
model saved to models_char_rnn/model.ckpt
1100/22300 (epoch 2), train_loss = 3.177, time/batch = 0.356
1200/22300 (epoch 2), train_loss = 2.993, time/batch = 0.330
1300/22300

12200/22300 (epoch 27), train_loss = 2.029, time/batch = 0.332
12300/22300 (epoch 27), train_loss = 1.952, time/batch = 0.333
12400/22300 (epoch 27), train_loss = 1.969, time/batch = 0.334
12500/22300 (epoch 28), train_loss = 1.992, time/batch = 0.334
12600/22300 (epoch 28), train_loss = 2.024, time/batch = 0.335
12700/22300 (epoch 28), train_loss = 1.999, time/batch = 0.331
12800/22300 (epoch 28), train_loss = 2.021, time/batch = 0.333
12900/22300 (epoch 28), train_loss = 1.998, time/batch = 0.334
13000/22300 (epoch 29), train_loss = 1.987, time/batch = 0.330
model saved to models_char_rnn/model.ckpt
13100/22300 (epoch 29), train_loss = 2.018, time/batch = 0.395
13200/22300 (epoch 29), train_loss = 2.020, time/batch = 0.335
13300/22300 (epoch 29), train_loss = 1.993, time/batch = 0.330
13400/22300 (epoch 30), train_loss = 2.044, time/batch = 0.330
13500/22300 (epoch 30), train_loss = 1.939, time/batch = 0.329
13600/22300 (epoch 30), train_loss = 2.041, time/batch = 0.357
13700/22300 (

In [10]:
# we're done with the model. the weights are now safe inside our storage
%ls {args.save_dir}

# so begone all the ops, graphs & variables!
# you may ask, why is this line needed? try commenting out the line and see what happens later in the sampling stage
tf.reset_default_graph()

chars_vocab.pkl  config.pkl                            model.ckpt-22299.index
checkpoint       model.ckpt-22299.data-00000-of-00001  model.ckpt-22299.meta


## Evalulation

<font color=red>**Your model could be evaluated without traning procedure,**</font> if you saved and loaded your model properly.

In [11]:
# load a bunch of libraries
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.contrib import legacy_seq2seq
import numpy as np
import argparse
import time
import os
from six.moves import cPickle
from six import text_type
import sys

# this module is from the utils.py file of this folder
# it handles loading texts to digits (aka. tokens) which are recognizable for the model
from utils import TextLoader

# this module is from the char_rnn.py file of this folder
# the task is implementing the CharRNN inside the class definition from this file
from char_rnn import Model

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# sampling definition for evaluation phase
# it uses the saved model and spit out some characters from the RNN model
def sample_eval(args):
    with open(os.path.join(args.save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(args.save_dir, 'chars_vocab.pkl'), 'rb') as f:
        chars, vocab = cPickle.load(f)
    #Use most frequent char if no prime is given
    if args.prime == '':
        args.prime = chars[0]
    model = Model(saved_args, training=False)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(args.save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            print(str(model.sample(sess, chars, vocab, args.n, args.prime)),'utf-8')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# argparsing for sampling from the model
parser_sample = argparse.ArgumentParser(
                   formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser_sample.add_argument('--save_dir', type=str, default='models_char_rnn',
                    help='model directory to store checkpointed models')
parser_sample.add_argument('-n', type=int, default=500,
                    help='number of characters to sample')
parser_sample.add_argument('--prime', type=text_type, default=u'',
                    help='prime text')
sys.argv = ['-f']
args_sample = parser_sample.parse_args()
args_sample

Namespace(n=500, prime='', save_dir='models_char_rnn')

In [13]:
# let's sample!
sample_eval(args_sample)
tf.reset_default_graph()

INFO:tensorflow:Restoring parameters from models_char_rnn/model.ckpt-22299
 vhuie rusf so powertte:
Yat: 'fint my gear tothor. Hodsers and inhars
hiy, then that to thas thim Tateroger neaves,
Tull dow meart our rerter the not for.
Thaw shaw, nepmouddlrind thou you havies day?
Fomer, retorth! whaols lare, atteetty thut, im nod hlourens his swithilsm-rion
The hay, and think wenling os naese do me. hooked!
It oull porning to the yraty peiths.
Hor reyy my mander?

Murdtanger:
My hwawweoch so datkot.

AAGENELUS:
Fhacan, wfadcy not.-'d son hung: by ip secessaor.

METCICHARD:
 utf-8


## Briefly summarize what & how you did, and why you did that way.
This is also for checking yourself if you really learned something from this assignment.

In case of Model class, It's about making LSTM architecture from cell. First of all, define LSTM cell and add dropout wrapper on each cell. Then, combine all of LSTM cells and make deep LSTM architecture. After defining input variables and other variables for inference, and also defining optimizers and loss function, one LSTM model is finally made. This LSTM Cell if for character RNN.

In sample function part, after feeding first character, I pick next word as stochastic. In other words, pick next word stochastically from weighted probability sum distribution.